# Keyword modeling with latent Dirichlet allocation

In [3]:
import gensim
import pandas as pd
import json

In [4]:
import pandas as pd
import json

articles = pd.read_pickle('ArticleMetadata.pkl')
articles.DatePublished = pd.to_datetime(articles.DatePublished)
articles.Tags = articles.Tags.map(lambda x: str(x))
articles.TagArray = articles.Tags.map(lambda x: x.split(','))
articles.TagArray[0]
articles.head(1)

,Url,Title,Tags,Topic,DatePublished,Abstract,FullText
ArticleId,,,,,,,
12897,/articles/amc-s-halt-and-catch-fire-is-capital...,"AMC’s ""Halt and Catch Fire"" Is Capitalism's Fi...","Capitalism,Competition,Property Rights,Entrepr...",Economics,2015-09-02 10:56:24,"""The show is a vibrant look at the early PC in...","""AMC's Halt and Catch Fire is a brilliant achi..."


In [5]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def StripHtml(html):
    return strip_tags(html)

print(StripHtml('<b>hello</b>'))
print(type(StripHtml('<b>hello</b>')))

hello
<class 'str'>


In [6]:
# prepare new field for LDA:    

articles["RawText"] = articles.FullText.map(lambda x: StripHtml(x))
articles["RawText"] = articles.RawText + ' ' +  articles.Title + ' ' + articles.Tags + ' ' + articles.Abstract  + ' ' + articles.Topic 
articles["RawText"].head()

ArticleId
12897    "AMC's Halt and Catch Fire is a brilliant achi...
58871    "Bush, Kennedy, Romney, Clinton, and, yes, eve...
58872    "How much government spending is enough, and h...
58873    "Progressive politicians have found a ripe old...
58874    "On Saturday night, millions of rich people pl...
Name: RawText, dtype: object

In [7]:
from gensim.models import Word2Vec

text= articles.RawText.dropna().map(lambda x: x.split())

In [8]:
model = Word2Vec(text, size=100, window=6, min_count=5, workers=4 )

In [10]:
model.most_similar(positive=['Trump'])

[('Sanders', 0.8593320846557617),
 ('Hillary', 0.7188717126846313),
 ('Bernie', 0.7109429836273193),
 ('Clinton', 0.706475555896759),
 ('Obama', 0.7017459869384766),
 ('Donald', 0.6958503723144531),
 ('Reich', 0.6757186651229858),
 ('Mr.', 0.6724414229393005),
 ('Castro', 0.6706552505493164),
 ('Trump,', 0.6692752242088318)]